# Calculate Quasi-Stationary Metrics of Your Data
Apply different stationarity definitions to your dataset and detect the most persistent cyclone tracks in time and space.

In [2]:
# Load python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import xarray as xr
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import time
from datetime import datetime
from matplotlib.path import Path
from scipy.io import savemat
from scipy.io import loadmat
from scipy.signal import convolve2d
from scipy import ndimage
from scipy.stats import rankdata
import multiprocessing
import os.path
import calendar

# Created files:
import QS_functions as fm
# !!! IMPORT QS_setup !

<class 'ModuleNotFoundError'>: No module named 'seaborn'

In [3]:
# definition of directory needed??

In [16]:
# Load df_crossers.mat
data = loadmat(in_dir+'df_crossers.mat')
print(data.keys())

df_crossers_ = pd.DataFrame({
    'id': data['id'].flatten(),
    'lon': data['lon'].flatten(),  # Use flatten() to convert to 1D array
    'lat': data['lat'].flatten(),
    'year': data['year'].flatten(),
    'month': data['month'].flatten(),
    'day': data['day'].flatten(),
    'time': data['time'].flatten(),
    'hPa': data['hPa'].flatten(),
    'medi_tracks': (data['medi_tracks'].flatten() == 1)
})
df_crossers

"\n# OTHERWISE: START FROM HERE\n# Load df_meditracks.mat and create dataset\ndata = loadmat(in_dir+'df_meditracks.mat')\nprint(data.keys())\n\ndf_meditracks_ = pd.DataFrame({\n    'id': data['id'].flatten(),\n    'lon': data['lon'].flatten(),  # Use flatten() to convert to 1D array\n    'lat': data['lat'].flatten(),\n    'year': data['year'].flatten(),\n    'month': data['month'].flatten(),\n    'day': data['day'].flatten(),\n    'time': data['time'].flatten(),\n    'hPa': data['hPa'].flatten(),\n    'medi_tracks': (data['medi_tracks'].flatten() == 1)\n})\ndf_meditracks_\n"

# Definitions of Stationarity

## Full-track Stationarity

### Median Velocity
- based on median speed along a cyclone's whole life cycle

In [18]:
# Calculate median velocity of each cyclone (df_crossers)                   #of all dist./hour we look for median
list_med_vel = []                                                      #create a list...
for c in np.unique(df_crossers['id'].values):                               #for as many unique id's as there are in df_crossers,
    id = df_crossers.loc[df_crossers['id'] == c]                            #DIFFERENT THAN FOR DF_TRACKS1: not all ids available, i.e. c =1, then no data stored as there exists no cyclone id 1 in df_crossers!
    list_medvel = []                                                   #and create a list...
    for t in range(len(id)-1):                                              #for as many t as there are in id-1; -1 to account for 0 at beginning of created t's -> LAST STEP NOT INCLUDED?
        medvel = fm.haversine(id.iloc[t,1], id.iloc[t,2],              #calculate all distances travelled per hour of each cyclone and store them as vel
                id.iloc[t+1,1], id.iloc[t+1,2])                 
        list_medvel.append(medvel)                                #add the velocities of each cyclone to the list
    med_vel = np.median(np.array(list_medvel))                    #look for the median velocity of each velocities of a cyclone
    list_med_vel.append(med_vel)                                  #add the medians to the list
median_vel = np.array(list_med_vel)                               #create an array of the medians       

#median_vel.tofile('/scratch2/mganci/for_marina/median_vel.txt', sep = ',')

In [20]:
# Calculate median velocity categories (10% fast, " around median, " slow)
# Create masks of median velocity categories for density maps
general_max_medv = np.amax(median_vel)                       #calculate the maximum of all median velocities (upper limit of range)
perc_90_medv = np.percentile(median_vel, 90)                 #calculate the 90 percentile (lower limit of "..10% fastest..")
general_fast = (median_vel[median_vel >= perc_90_medv])      #calculate the fastest 10%
mask_fast = median_vel >= perc_90_medv                       #calculate mask

general_median_medv = np.median(median_vel)                  #calculate the median of all median velocities
perc_45_medv = np.percentile(median_vel, 45)                 #calculate the 45 and 55 percentile (lower and upper limit of"..10% around the median..")
perc_55_medv = np.percentile(median_vel, 55)
general_middle = (median_vel[(median_vel>perc_45_medv) & (median_vel<perc_55_medv)]) #calculate the 10% around the general median
mask_middle = (median_vel>perc_45_medv) & (median_vel<perc_55_medv)   #calculate mask

general_min_medv = np.amin(median_vel)                       #calculate the minimum of all median velocities (lower limit of range)
perc_10_medv = np.percentile(median_vel, 10)                 #calculate the 10 percentile (upper limit of "..10% slowest..")
general_slow = (median_vel[median_vel <= perc_10_medv])      #calculate the slowest 10%
mask_slow = median_vel <= perc_10_medv                       #calculate mask

In [21]:
# Connect masks with cyclone id's in df_crossers, so that all info of cyclones is kept
id_fast = (np.arange(len(median_vel))+1)[mask_fast]                 #create id-vector of all id's of median_vel machting the mask
id_middle = (np.arange(len(median_vel))+1)[mask_middle]                 #create id-vector of all id's of median_vel machting the mask
id_slow = (np.arange(len(median_vel))+1)[mask_slow]                 #create id-vector of all id's of median_vel machting the mask

tracks_fast = df_crossers.loc[df_crossers['id'].isin(id_fast)]     #show me all cyclone id's in df_crossers which correspond to the above id vector
tracks_middle = df_crossers.loc[df_crossers['id'].isin(id_middle)]     #show me all cyclone id's in df_crossers which correspond to the above id vector
tracks_slow = df_crossers.loc[df_crossers['id'].isin(id_slow)]     #show me all cyclone id's in df_crossers which correspond to the above id vector

#tracks_fast.to_csv('/scratch2/mganci/for_marina/tracks_fast.txt', header=True, sep='\t')                   #to export to textfile and read in another coding-file
#tracks_middle.to_csv('/scratch2/mganci/for_marina/tracks_middle.txt', header=True, sep='\t')
#tracks_slow.to_csv('/scratch2/mganci/for_marina/tracks_slow.txt', header=True, sep='\t')
#tracks_slow.to_csv('/home/lbaechtold/masterthesis/data/median_stationarity/tracks_slow.csv', header=True, index=False)

#### Preparation for QS table

In [22]:
# 1. Check len
len(median_vel)

2377

In [23]:
# 2. All Quantiles
# use rankdata to calculate ranks, with method='average' to handle ties
ranks = rankdata(median_vel, method='ordinal', nan_policy='omit')

# calculate quantile ranks
FT_MED_VEL_q = np.round(
                    np.where(
                        np.isnan(median_vel), np.nan,
                            (ranks - 1) / (len(median_vel[~np.isnan(median_vel)]) - 1)
                    ),
                    3
                )

# this array now contains the quantile ranks
FT_MED_VEL_q

array([0.831, 0.836, 0.745, ..., 0.476, 0.889, 0.078])

In [24]:
len(FT_MED_VEL_q)

2377

In [25]:
# 3. All Categories (0: Nan, 1: <10%, 2: 45-55%, 3: >90%)
# Assign categories
FT_MED_VEL_c = np.zeros_like(FT_MED_VEL_q)

FT_MED_VEL_c[FT_MED_VEL_q <= .1] = 1
FT_MED_VEL_c[(FT_MED_VEL_q >= .45) & (FT_MED_VEL_q <= .55)] = 2
FT_MED_VEL_c[FT_MED_VEL_q >= .9] = 3

FT_MED_VEL_c[np.isnan(FT_MED_VEL_q)] = np.nan
FT_MED_VEL_c

array([0., 0., 0., ..., 2., 0., 1.])

In [26]:
# 3.1. sanity test: random selection to see whether it worked
print(FT_MED_VEL_q[2000:2100])
print(FT_MED_VEL_c[2000:2100])

[0.519 0.136 0.392 0.24  0.351 0.688 0.001 0.39  0.9   0.377 0.656 0.451
, 0.935 0.256 0.067 0.742 0.381 0.274 0.279 0.103 0.516 0.457 0.36  0.806
, 0.604 0.913 0.328 0.902 0.258 0.55  0.611 0.266 0.355 0.871 0.262 0.911
, 0.671 0.011 0.412 0.429 0.774 0.234 0.528 0.114 0.826 0.71  0.777 0.587
, 0.578 0.501 0.601 0.876 0.552 0.463 0.109 0.454 0.127 0.947 0.462 0.923
, 0.354 0.126 0.269 0.611 0.12  0.433 0.58  0.162 0.422 0.404 0.65  0.025
, 0.286 0.038 0.893 0.304 0.811 0.082 0.028 0.029 0.108 0.134 0.006 0.798
, 0.866 0.479 0.644 0.734 0.748 0.593 0.359 0.949 0.601 0.338 0.476 0.505
, 0.185 0.706 0.305 0.428]
,[2. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 2. 3. 0. 1. 0. 0. 0. 0. 0. 2. 2. 0. 0.
, 0. 3. 0. 3. 0. 2. 0. 0. 0. 0. 0. 3. 0. 1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.
, 0. 2. 0. 0. 0. 2. 0. 2. 0. 3. 2. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
, 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 0. 3. 0. 0. 2. 2.
, 0. 0. 0. 0.]


In [27]:
# 4. Add Columns to Table
new_cols_medvel = pd.DataFrame([FT_MED_VEL_c, FT_MED_VEL_q, median_vel]).T
new_cols_medvel.columns = ['FT_MED_VEL_c', 'FT_MED_VEL_q', 'FT_MED_VEL_v']
# new_cols_medvel.index = df_crossers.index
new_cols_medvel

,FT_MED_VEL_c,FT_MED_VEL_q,FT_MED_VEL_v
0,0.0,0.831,31.008022
1,0.0,0.836,31.154744
2,0.0,0.745,27.322200
3,3.0,0.953,40.211951
4,0.0,0.659,24.748190
...,...,...,...
2372,0.0,0.716,26.546358
2373,3.0,0.917,36.021598
2374,2.0,0.476,20.293505
2375,0.0,0.889,33.954863


In [28]:
# 5. QS Table non-complete
# Copy df_crossers to a new df
df_crossers_medvel = df_crossers.copy()

# Get unique ids in df_crossers
unique_ids = df_crossers['id'].unique()

# Iterate over each unique id in df_crossers
for i, unique_id in enumerate(unique_ids):
    # Get the corresponding row from new_cols_medvel
    if i < len(new_cols_medvel):
        matching_row = new_cols_medvel.iloc[i]
    else:
        matching_row = new_cols_medvel.iloc[-1]  # Use the last row if there are more ids than rows
    
    # Assign the values from the matching row to all rows with the current id in df_crossers_medvel
    df_crossers_medvel.loc[df_crossers_medvel['id'] == unique_id, 'FT_MED_VEL_c'] = matching_row['FT_MED_VEL_c']
    df_crossers_medvel.loc[df_crossers_medvel['id'] == unique_id, 'FT_MED_VEL_q'] = matching_row['FT_MED_VEL_q']
    df_crossers_medvel.loc[df_crossers_medvel['id'] == unique_id, 'FT_MED_VEL_v'] = matching_row['FT_MED_VEL_v']

# Display the result
df_crossers_medvel

,id,lon,lat,year,month,day,time,hPa,medi_track,FT_MED_VEL_c,FT_MED_VEL_q,FT_MED_VEL_v
229,4,0.960,40.700,1979,1,8,18,1016.93,True,0.0,0.831,31.008022
230,4,1.080,40.761,1979,1,8,19,1016.93,True,0.0,0.831,31.008022
231,4,1.300,40.746,1979,1,8,20,1016.64,True,0.0,0.831,31.008022
232,4,1.624,40.699,1979,1,8,21,1016.35,True,0.0,0.831,31.008022
233,4,1.986,40.693,1979,1,8,22,1015.86,True,0.0,0.831,31.008022
...,...,...,...,...,...,...,...,...,...,...,...,...
267292,3807,36.114,44.208,2020,9,8,2,1015.46,True,1.0,0.078,10.430317
267293,3807,36.228,44.162,2020,9,8,3,1015.79,True,1.0,0.078,10.430317
267294,3807,36.323,44.163,2020,9,8,4,1016.16,True,1.0,0.078,10.430317
267295,3807,36.414,44.197,2020,9,8,5,1016.56,True,1.0,0.078,10.430317


### Total Distance (Path Stationarity by Aregger, 2021)
- based on maximum distance that a storm can travel over whole lifetime
- calculated by summing up the distances between each observational timestep

In [29]:
## Total distance (travelled during total lifetime) of each cyclone (df_crossers)
list_tot_dist = []                                              #create a list of...
for c in np.unique(df_crossers['id'].values):   #for as many c as there are in range(..),
    id = df_crossers.loc[df_crossers['id'] == c]                  #store all the id's in df_crossers in id
    list_dist = []                                              #and create a list of...
    for t in range(len(id)-1):                                  #for as many t as there are in id-1; -1 to exclude 0 at beginning of created t's
        dist = fm.haversine(id.iloc[t,1], id.iloc[t,2],            #calculate all distances travelled of each cyclone and store them as dist
                id.iloc[t+1,1], id.iloc[t+1,2])                 
        list_dist.append(dist)                                  #add the distances to the list
    tot_dist = np.sum(np.array(list_dist))                      #sum all the distances of a cyclone up to a total one
    list_tot_dist.append(tot_dist)                              #add the total distances to the list
all_dist = np.array(list_tot_dist)                              #create an array of the distances

In [31]:
#Distance categories (10% largest, " around median, " smallest)
#Create masks of distance categories for density maps
general_max_d = np.amax(all_dist)                                                             #calculate the maximum of all distances (upper limit of range)
perc_90_d = np.percentile(all_dist, 90)                                                       #calculate the 90 percentile (lower limit of "..10% longest..")
general_long = (all_dist[all_dist >= perc_90_d])                                    #calculate the longest 10%
mask_long = all_dist >= perc_90_d                                                        #calculate mask

general_median_d = np.median(all_dist)                                                        #calculate the median of all distances
perc_45_d = np.percentile(all_dist, 45)                                                       #calculate the 45 and 55 percentile (lower and upper limit of"..10% around the median..")
perc_55_d = np.percentile(all_dist, 55)
general_midi = (all_dist[(all_dist>perc_45_d) & (all_dist<perc_55_d)])    #calculate the 10% around the general median
mask_midi = (all_dist>perc_45_d) & (all_dist<perc_55_d)                        #calculate mask

general_min_d = np.amin(all_dist)                                                             #calculate the minimum of all distances (lower limit of range)
perc_10_d = np.percentile(all_dist, 10)                                                       #calculate the 10 percentile (upper limit of "..10% shortest..")
general_short = (all_dist[all_dist <= perc_10_d])                                   #calculate the shortest 10%
mask_short = all_dist <= perc_10_d                                                       #calculate mask

In [33]:
#Connect masks with cyclone id's in df_crossers, so that all info of cyclones is available
id_long = (np.arange(len(all_dist))+1)[mask_long]                 #create id-vector of all id's of all_dist machting the mask
id_midi = (np.arange(len(all_dist))+1)[mask_midi]                 #create id-vector of all id's of all_dist machting the mask
id_short = (np.arange(len(all_dist))+1)[mask_short]                 #create id-vector of all id's of all_dist machting the mask

tracks_long = df_crossers.loc[df_crossers['id'].isin(id_long)]     #show me all cyclone id's in df_crossers which correspond to the above id vector
tracks_midi = df_crossers.loc[df_crossers['id'].isin(id_midi)]     #show me all cyclone id's in df_crossers which correspond to the above id vector
tracks_short = df_crossers.loc[df_crossers['id'].isin(id_short)]     #show me all cyclone id's in df_crossers which correspond to the above id vector

#tracks_long.to_csv('/scratch2/mganci/for_marina/tracks_long_Aregger-totdist.txt', header=True, sep='\t')
#tracks_midi.to_csv('/scratch2/mganci/for_marina/tracks_midi_Aregger-totdist.txt', header=True, sep='\t')
#tracks_short.to_csv('/scratch2/mganci/for_marina/tracks_short_Aregger-totdis.txt', header=True, sep='\t')

#### Preparation for QS table

In [34]:
# 1. Check len
len(all_dist)

2377

In [35]:
# 2. All Quantiles
# use rankdata to calculate ranks, with method='average' to handle ties
ranks = rankdata(all_dist, method='ordinal', nan_policy='omit')

# calculate quantile ranks
FT_TOT_DIST_q = np.round(
                    np.where(
                        np.isnan(all_dist), np.nan,
                            (ranks - 1) / (len(all_dist[~np.isnan(all_dist)]) - 1)
                    ),
                    3
                )

# this array now contains the quantile ranks
FT_TOT_DIST_q

array([0.391, 0.404, 0.947, ..., 0.259, 0.564, 0.25 ])

In [36]:
# 3. All Categories (0: Nan, 1: <10%, 2: 45-55%, 3: >90%)
# Assign categories
FT_TOT_DIST_c = np.zeros_like(FT_TOT_DIST_q)

FT_TOT_DIST_c[FT_TOT_DIST_q <= .1] = 1
FT_TOT_DIST_c[(FT_TOT_DIST_q >= .45) & (FT_TOT_DIST_q <= .55)] = 2
FT_TOT_DIST_c[FT_TOT_DIST_q >= .9] = 3

FT_TOT_DIST_c[np.isnan(FT_TOT_DIST_q)] = np.nan
FT_TOT_DIST_c

array([0., 0., 3., ..., 0., 0., 0.])

In [37]:
# 3.1. sanity test: random selection to see whether it worked
print(FT_TOT_DIST_q[2000:2100])
print(FT_TOT_DIST_c[2000:2100])

[0.248 0.485 0.522 0.851 0.138 0.673 0.031 0.75  0.622 0.362 0.673 0.729
, 0.926 0.365 0.333 0.239 0.371 0.245 0.695 0.475 0.181 0.661 0.186 0.344
, 0.342 0.928 0.757 0.403 0.214 0.558 0.72  0.533 0.281 0.833 0.062 0.711
, 0.811 0.004 0.869 0.423 0.537 0.822 0.332 0.957 0.7   0.149 0.576 0.215
, 0.874 0.95  0.691 0.971 0.114 0.358 0.178 0.26  0.231 0.83  0.806 0.997
, 0.091 0.389 0.614 0.883 0.075 0.537 0.923 0.341 0.463 0.777 0.408 0.328
, 0.334 0.04  0.843 0.581 0.594 0.153 0.367 0.003 0.201 0.286 0.004 0.587
, 0.327 0.35  0.147 0.261 0.808 0.488 0.127 0.702 0.501 0.103 0.903 0.417
, 0.152 0.853 0.665 0.613]
,[0. 2. 2. 0. 0. 0. 1. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0.
, 0. 3. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 1. 0. 0. 2. 0. 0. 3. 0. 0. 0. 0.
, 0. 3. 0. 3. 0. 0. 0. 0. 0. 0. 0. 3. 1. 0. 0. 0. 1. 2. 3. 0. 2. 0. 0. 0.
, 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 3. 0.
, 0. 0. 0. 0.]


In [38]:
# 4. Add Columns to Table
new_cols_totdist = pd.DataFrame([FT_TOT_DIST_c, FT_TOT_DIST_q, all_dist]).T
new_cols_totdist.columns = ['FT_TOT_DIST_c', 'FT_TOT_DIST_q', 'FT_TOT_DIST_v']
new_cols_totdist.reindex(df_crossers.index)

,FT_TOT_DIST_c,FT_TOT_DIST_q,FT_TOT_DIST_v
229,0.0,0.175,921.937971
230,3.0,0.949,3513.441774
231,3.0,0.920,3222.112009
232,0.0,0.250,1069.360654
233,0.0,0.209,988.758648
...,...,...,...
267292,NaN,NaN,NaN
267293,NaN,NaN,NaN
267294,NaN,NaN,NaN
267295,NaN,NaN,NaN


In [39]:
# 5. QS Table non-complete
# Copy df_crossers to a new df
df_crossers_medvel_totdist = df_crossers_medvel.copy()

# Get unique ids in df_crossers
unique_ids = df_crossers['id'].unique()

# Iterate over each unique id in df_crossers
for i, unique_id in enumerate(unique_ids):
    # Get the corresponding row from new_cols_totdist
    if i < len(new_cols_totdist):
        matching_row = new_cols_totdist.iloc[i]
    else:
        matching_row = new_cols_totdist.iloc[-1]  # Use the last row if there are more ids than rows
    
    # Assign the values from the matching row to all rows with the current id in df_crossers_medvel_totdist
    df_crossers_medvel_totdist.loc[df_crossers_medvel_totdist['id'] == unique_id, 'FT_TOT_DIST_c'] = matching_row['FT_TOT_DIST_c']
    df_crossers_medvel_totdist.loc[df_crossers_medvel_totdist['id'] == unique_id, 'FT_TOT_DIST_q'] = matching_row['FT_TOT_DIST_q']
    df_crossers_medvel_totdist.loc[df_crossers_medvel_totdist['id'] == unique_id, 'FT_TOT_DIST_v'] = matching_row['FT_TOT_DIST_v']

# Display the result
df_crossers_medvel_totdist

,id,lon,lat,year,month,day,time,hPa,medi_track,FT_MED_VEL_c,FT_MED_VEL_q,FT_MED_VEL_v,FT_TOT_DIST_c,FT_TOT_DIST_q,FT_TOT_DIST_v
229,4,0.960,40.700,1979,1,8,18,1016.93,True,0.0,0.831,31.008022,0.0,0.391,1355.115214
230,4,1.080,40.761,1979,1,8,19,1016.93,True,0.0,0.831,31.008022,0.0,0.391,1355.115214
231,4,1.300,40.746,1979,1,8,20,1016.64,True,0.0,0.831,31.008022,0.0,0.391,1355.115214
232,4,1.624,40.699,1979,1,8,21,1016.35,True,0.0,0.831,31.008022,0.0,0.391,1355.115214
233,4,1.986,40.693,1979,1,8,22,1015.86,True,0.0,0.831,31.008022,0.0,0.391,1355.115214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267292,3807,36.114,44.208,2020,9,8,2,1015.46,True,1.0,0.078,10.430317,0.0,0.250,1068.240564
267293,3807,36.228,44.162,2020,9,8,3,1015.79,True,1.0,0.078,10.430317,0.0,0.250,1068.240564
267294,3807,36.323,44.163,2020,9,8,4,1016.16,True,1.0,0.078,10.430317,0.0,0.250,1068.240564
267295,3807,36.414,44.197,2020,9,8,5,1016.56,True,1.0,0.078,10.430317,0.0,0.250,1068.240564


## Along Track Stationarity

### 12-hour Distance
1. For each ID, calculate all dist btw 2 points and take sum over 12 consecutive distances (13 points = 12 individual distances = 12hrs). Repeat for all points (as starting point) and all IDs.
3. Look for the 10% largest, mid, short 12-hour Distances.
4. Plot the coordinates of the reference-points.


In [48]:
# 1. For each ID, calculate 12-hour Distances for 13 consecutive points (=12hrs). Repeat for all points being once a reference point.
def calculate_12hrs(df):
    list_dist_12hrs = []  # This will store the list_dist_12hrs as a nested list
    all_dist_12hrs = []  # This will store the total 12-hour Distances for each reference point

    for id, group in df.groupby('id'):  # Group by ID to handle each ID separately
        lats = group['lat'].values  # Extract latitudes and longitudes
        lons = group['lon'].values

        for i in range(len(group)):  # Iterate over each point in the group
            # Skip if fewer than 13 points follow the reference point
            if i + 12 >= len(group):
                continue
            
            # List to store distances for the current window (from point i to i+12)
            dist_per_ref = []

            # Calculate the total distance from point i to the next 12 points
            for j in range(i, i + 12):  # Iterate through the next 12 consecutive points
                distance = fm.haversine(lons[j], lats[j], lons[j + 1], lats[j + 1])  # Distance from point j to j+1
                dist_per_ref.append(distance)

            # Sum the distances for the total distance of this window of 13 points
            dist_12hrs = sum(dist_per_ref)
            all_dist_12hrs.append(dist_12hrs)

            # Append list_dist_12hrs in the format: [id, lon, lat, total_dist_to_next_13]
            list_dist_12hrs.append([id, lons[i], lats[i], dist_12hrs])

    return list_dist_12hrs, all_dist_12hrs

# Test for df_crossers
list_dist_12hrs, all_dist_12hrs = calculate_12hrs(df_crossers)

In [49]:
# 2. Look for the 10% largest, mid, short 12-hour Distances within all_dist_12hrs and get the coordinates of the ref points
# 2.1. Calculate percentiles
general_max_12 = np.amax(all_dist_12hrs)                       #calculate the maximum of all 12-hour Distances (upper limit of range)
perc_90_12 = np.percentile(all_dist_12hrs, 90)                 #calculate the 90 percentile (lower limit of "..10% longest..")

general_median_12 = np.median(all_dist_12hrs)                  #calculate the median of all 12-hour Distances
perc_45_12 = np.percentile(all_dist_12hrs, 45)                 #calculate the 45 and 55 percentile (lower and upper limit of"..10% around the median..")
perc_55_12 = np.percentile(all_dist_12hrs, 55)

general_min_12 = np.amin(all_dist_12hrs)                       #calculate the minimum of all 12-hour Distances (lower limit of range)
perc_10_12 = np.percentile(all_dist_12hrs, 10)                 #calculate the 10 percentile (upper limit of "..10% shor..")


In [50]:
# 2.2. Define 10% categories and ref. points
all_long12 = []
all_midi12 = []
all_short12 = []

ref_long_12 = []
ref_midi_12 = []
ref_short_12 = []

# Loop over each result in the distances list
for result in list_dist_12hrs:
    ref_lon12, ref_lat12, dist_12hrs = result[1], result[2], result[3]
    
    # 10% longst
    if dist_12hrs >= perc_90_12:
        all_long12.append(dist_12hrs)
        ref_long_12.append([ref_lon12, ref_lat12])
    
    # Midi (45th to 55th percentiles)
    if perc_45_12 <= dist_12hrs <= perc_55_12:
        all_midi12.append(dist_12hrs)
        ref_midi_12.append([ref_lon12, ref_lat12])
    
    # 10% Shortest
    if dist_12hrs <= perc_10_12:
        all_short12.append(dist_12hrs)
        ref_short_12.append([ref_lon12, ref_lat12])

In [51]:
# 3. Connect coordinates with all info of cyclones
tracks_long12_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_long_12])]
tracks_midi12_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_midi_12])]
tracks_short12_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_short_12])]

In [52]:
tracks_long12_ref

,id,lon,lat,year,month,day,time,hPa,medi_track
283,5,23.999,40.097,1979,1,13,6,998.40,True
284,5,24.068,40.100,1979,1,13,7,998.65,True
285,5,24.259,40.225,1979,1,13,8,998.57,True
286,5,24.612,40.447,1979,1,13,9,998.66,True
287,5,25.063,40.713,1979,1,13,10,997.74,True
...,...,...,...,...,...,...,...,...,...
267004,3802,13.939,39.159,2020,12,27,1,1006.62,True
267035,3803,24.247,40.004,2020,2,5,7,989.47,True
267036,3803,24.444,39.859,2020,2,5,8,989.43,True
267151,3806,10.720,38.010,2020,3,4,2,1004.28,True


#### Preparation for QS Table

In [53]:
# 1. All Values
full_dist_12hrs_values = []
for ID_unique in np.unique(df_crossers.id.values):                       # for each id in df_crossers
    x = np.append(
            np.round(
                np.array(
                    [value[-1:][0] for value in                           # select 4th item, i.e. value
                     [i for i in list_dist_12hrs if i[0] == ID_unique]]), # select id
                3                                                         # round values up to 3rd decimal place
            ),
            [np.nan] * 12                                                 # append 12 np.nans at the end of cyclone
        )
    full_dist_12hrs_values.append(x)

AT_12h_DIST_v = np.concatenate(full_dist_12hrs_values)
AT_12h_DIST_v

array([293.162, 313.308, 321.903, ...,     nan,     nan,     nan])

In [54]:
# 1.1. sanity check: compare length of array to make sure same
print(len(df_crossers))
print(len(AT_12h_DIST_v))

173790
,173790


In [55]:
# 2. All Quantiles
# use rankdata to calculate ranks, with method='average' to handle ties
ranks = rankdata(AT_12h_DIST_v, method='ordinal', nan_policy='omit')

# calculate quantile ranks
AT_12h_DIST_q = np.round(
                    np.where(
                        np.isnan(AT_12h_DIST_v), np.nan,
                            (ranks - 1) / (len(AT_12h_DIST_v[~np.isnan(AT_12h_DIST_v)]) - 1)
                    ),
                    3
                )

# this array now contains the quantile ranks
AT_12h_DIST_q

array([0.585, 0.628, 0.645, ...,   nan,   nan,   nan])

In [56]:
# 3. All Categories (0: Nan, 1: <10%, 2: 45-55%, 3: >90%)
# Assign categories
AT_12h_DIST_c = np.zeros_like(AT_12h_DIST_q)

AT_12h_DIST_c[AT_12h_DIST_q <= .1] = 1
AT_12h_DIST_c[(AT_12h_DIST_q >= .45) & (AT_12h_DIST_q <= .55)] = 2
AT_12h_DIST_c[AT_12h_DIST_q >= .9] = 3

AT_12h_DIST_c[np.isnan(AT_12h_DIST_q)] = np.nan
AT_12h_DIST_c

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [57]:
# 3.1. sanity test: random selection to see whether it worked
print(AT_12h_DIST_q[2000:2100])
print(AT_12h_DIST_c[2000:2100])

[0.15  0.131 0.106 0.109 0.112 0.106 0.099 0.099 0.095 0.082 0.065 0.06
, 0.059 0.057 0.063 0.056 0.039 0.022 0.018 0.024 0.045 0.071 0.097 0.12
, 0.165 0.216 0.245 0.242 0.282 0.349 0.399 0.42  0.41  0.389 0.382 0.38
, 0.368 0.375 0.418 0.489 0.505 0.479 0.463 0.518 0.658 0.799   nan   nan
,   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan 0.331 0.312
, 0.276 0.231 0.198 0.183 0.169 0.142 0.132 0.173 0.264 0.357 0.412 0.425
, 0.435 0.465 0.501 0.532 0.56  0.593 0.608 0.587 0.531 0.477 0.457 0.477
, 0.502 0.51  0.511 0.515 0.525 0.536 0.55  0.559 0.561 0.56  0.561 0.563
, 0.563 0.563 0.566 0.575]
,[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
,  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
,  0.  0.  0.  2.  2.  2.  2.  2.  0.  0. nan nan nan nan nan nan nan nan
, nan nan nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
,  0.  2.  2.  2.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
,  2.  0.  0.  

In [58]:
# 4. Add Columns to Table
new_cols_12 = pd.DataFrame([AT_12h_DIST_c, AT_12h_DIST_q, AT_12h_DIST_v]).T
new_cols_12.columns = ['AT_12h_DIST_c', 'AT_12h_DIST_q', 'AT_12h_DIST_v']
new_cols_12.index = df_crossers.index

In [59]:
# 5. QS Table non-complete
df_crossers_medvel_totdist_12 = pd.concat([df_crossers_medvel_totdist, new_cols_12], axis=1)
df_crossers_medvel_totdist_12

,id,lon,lat,year,month,day,time,hPa,medi_track,FT_MED_VEL_c,FT_MED_VEL_q,FT_MED_VEL_v,FT_TOT_DIST_c,FT_TOT_DIST_q,FT_TOT_DIST_v,AT_12h_DIST_c,AT_12h_DIST_q,AT_12h_DIST_v
229,4,0.960,40.700,1979,1,8,18,1016.93,True,0.0,0.831,31.008022,0.0,0.391,1355.115214,0.0,0.585,293.162
230,4,1.080,40.761,1979,1,8,19,1016.93,True,0.0,0.831,31.008022,0.0,0.391,1355.115214,0.0,0.628,313.308
231,4,1.300,40.746,1979,1,8,20,1016.64,True,0.0,0.831,31.008022,0.0,0.391,1355.115214,0.0,0.645,321.903
232,4,1.624,40.699,1979,1,8,21,1016.35,True,0.0,0.831,31.008022,0.0,0.391,1355.115214,0.0,0.640,319.393
233,4,1.986,40.693,1979,1,8,22,1015.86,True,0.0,0.831,31.008022,0.0,0.391,1355.115214,0.0,0.629,313.970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267292,3807,36.114,44.208,2020,9,8,2,1015.46,True,1.0,0.078,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN
267293,3807,36.228,44.162,2020,9,8,3,1015.79,True,1.0,0.078,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN
267294,3807,36.323,44.163,2020,9,8,4,1016.16,True,1.0,0.078,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN
267295,3807,36.414,44.197,2020,9,8,5,1016.56,True,1.0,0.078,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN


### Radial Distance
1. For each ID, calculate distance from reference point to 12 consecutive points (13 points = 12 distances = 12 hrs). Start at point 0 = reference, Stop at last point followed by 12 other points. Repeat for all points of an ID.
2. Look for the 10% of reference points with longest, mid and smallest radial-distances
3. Find the coord. of the reference points.

In [60]:
# 1. For each ID, calculate distance from reference point to 12 consecutive points, starting always at reference point. Repeat for all points and all ids.
def calculate_radial(df):
    list_dist_rad = []                                                        # This will store the list_dist_rad as a nested list
    all_dist_rad = []                                                   # This will store only the sum of distances (dist_sums_for_ref) for all points
    for id, group in df.groupby('id'):                                  # Group by ID to handle each ID separately
        lats = group['lat'].values                                      # Extract latitudes and longitudes
        lons = group['lon'].values

        for i in range(len(group)):                                     # Calculate distances for each reference point, up to the 12th point by looping trough all points
            dist_for_ref = []
            if i + 12 >= len(group):                                    # Skip if fewer than 12 points follow the reference point
                continue
            for j in range(i + 1, i + 13):                              # Calculate distance from point i to the next 12 points
                rad_dist = fm.haversine(lons[i], lats[i], lons[j], lats[j])
                dist_for_ref.append(rad_dist)
            dist_sums_for_ref = sum(dist_for_ref)                     # Sum the distances for the reference point
            all_dist_rad.append(dist_sums_for_ref)
            list_dist_rad.append([id, lons[i], lats[i], dist_sums_for_ref])        # Store in nested list with values...
    return list_dist_rad, all_dist_rad

# Test for df_crossers
list_dist_rad, all_dist_rad = calculate_radial(df_crossers)

In [61]:
# 2. Look for the 10% largest, mid, short all_dist_rad and get the lat/lon of their ref. point
# 2.1. Calculate percentiles
general_max_rad = np.amax(all_dist_rad)                       #calculate the maximum of all rad-distances (upper limit of range)
perc_90_rad = np.percentile(all_dist_rad, 90)                 #calculate the 90 percentile (lower limit of "..10% longest..")

general_median_rad = np.median(all_dist_rad)                  #calculate the median of all rad-distances
perc_45_rad = np.percentile(all_dist_rad, 45)                 #calculate the 45 and 55 percentile (lower and upper limit of"..10% around the median..")
perc_55_rad = np.percentile(all_dist_rad, 55)

general_min_rad = np.amin(all_dist_rad)                       #calculate the minimum of all rad-distances (lower limit of range)
perc_10_rad = np.percentile(all_dist_rad, 10)                 #calculate the 10 percentile (upper limit of "..10% shor..")


In [62]:
# 2.2. Define 10% categories and ref. points
all_grad = []
all_midirad = []
all_shortrad = []

ref_long_rad = []
ref_midi_rad = []
ref_short_rad = []

# Loop over each result in the distances list
for result in list_dist_rad:
    ref_lonrad, ref_latrad, dist_sums_for_ref = result[1], result[2], result[3]
    
    # 10% gst
    if dist_sums_for_ref >= perc_90_rad:
        all_grad.append(dist_sums_for_ref)
        ref_long_rad.append([ref_lonrad, ref_latrad])
    
    # Midi (45th to 55th percentiles)
    if perc_45_rad <= dist_sums_for_ref <= perc_55_rad:
        all_midirad.append(dist_sums_for_ref)
        ref_midi_rad.append([ref_lonrad, ref_latrad])
    
    # 10% Shortest
    if dist_sums_for_ref <= perc_10_rad:
        all_shortrad.append(dist_sums_for_ref)
        ref_short_rad.append([ref_lonrad, ref_latrad])

In [63]:
# 3. Connect coordinates with all info of cyclones
tracks_longrad_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_long_rad])]
tracks_midirad_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_midi_rad])]
tracks_shortrad_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_short_rad])]

#### Preparation for QS table

In [64]:
# 1. All Values
full_dist_rad_values = []
for ID_unique in np.unique(df_crossers.id.values):                       # for each id in df_crossers
    x = np.append(
            np.round(
                np.array(
                    [value[-1:][0] for value in                           # select 4th item, i.e. value
                     [i for i in list_dist_rad if i[0] == ID_unique]]), # select id
                3                                                         # round values up to 3rd decimal place
            ),
            [np.nan] * 12                                                 # append 12 np.nans at the end of cyclone
        )
    full_dist_rad_values.append(x)

AT_RAD_DIST_v = np.concatenate(full_dist_rad_values)
AT_RAD_DIST_v

array([1554.441, 1710.421, 1779.949, ...,      nan,      nan,      nan])

In [65]:
# 1.1. sanity check: compare length of array to make sure same
print(len(df_crossers))
print(len(AT_RAD_DIST_v))

173790
,173790


In [66]:
# 2. All Quantiles
# use rankdata to calculate ranks, with method='average' to handle ties
ranks = rankdata(AT_RAD_DIST_v, method='ordinal', nan_policy='omit')

# calculate quantile ranks
AT_RAD_DIST_q = np.round(
                    np.where(
                        np.isnan(AT_RAD_DIST_v), np.nan,
                            (ranks - 1) / (len(AT_RAD_DIST_v[~np.isnan(AT_RAD_DIST_v)]) - 1)
                    ),
                    3
                )

# this array now contains the quantile ranks
AT_RAD_DIST_q

array([0.513, 0.566, 0.588, ...,   nan,   nan,   nan])

In [67]:
# 3. All Categories (0: Nan, 1: <10%, 2: 45-55%, 3: >90%)
# Assign categories
AT_RAD_DIST_c = np.zeros_like(AT_RAD_DIST_q)

AT_RAD_DIST_c[AT_RAD_DIST_q <= .1] = 1
AT_RAD_DIST_c[(AT_RAD_DIST_q >= .45) & (AT_RAD_DIST_q <= .55)] = 2
AT_RAD_DIST_c[AT_RAD_DIST_q >= .9] = 3

AT_RAD_DIST_c[np.isnan(AT_RAD_DIST_q)] = np.nan
AT_RAD_DIST_c

array([ 2.,  0.,  0., ..., nan, nan, nan])

In [68]:
# 3.1. sanity test: random selection to see whether it worked
print(AT_RAD_DIST_q[2000:2100])
print(AT_RAD_DIST_c[2000:2100])

[0.082 0.08  0.085 0.077 0.036 0.008 0.02  0.044 0.05  0.041 0.038 0.054
, 0.086 0.131 0.164 0.158 0.114 0.062 0.024 0.011 0.013 0.025 0.04  0.054
, 0.075 0.104 0.137 0.178 0.223 0.246 0.235 0.194 0.133 0.103 0.111 0.129
, 0.175 0.272 0.391 0.471 0.481 0.453 0.425 0.431 0.499 0.61    nan   nan
,   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan 0.458 0.447
, 0.403 0.336 0.289 0.273 0.271 0.262 0.236 0.194 0.143 0.121 0.128 0.156
, 0.199 0.253 0.319 0.388 0.459 0.512 0.513 0.424 0.294 0.267 0.342 0.425
, 0.468 0.478 0.483 0.501 0.53  0.564 0.594 0.61  0.612 0.61  0.61  0.609
, 0.603 0.596 0.592 0.594]
,[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  1.
,  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
,  0.  0.  0.  2.  2.  2.  0.  0.  2.  0. nan nan nan nan nan nan nan nan
, nan nan nan nan  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
,  0.  0.  0.  0.  2.  2.  2.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  0.
,  0.  0.  0

In [69]:
# 4. Add Columns to QS Table
new_cols_rad = pd.DataFrame([AT_RAD_DIST_c, AT_RAD_DIST_q, AT_RAD_DIST_v]).T
new_cols_rad.columns = ['AT_RAD_DIST_c', 'AT_RAD_DIST_q', 'AT_RAD_DIST_v']
new_cols_rad.index = df_crossers.index

In [70]:
# 5. QS Table non-complete
df_crossers_medvel_totdist_12_rad = pd.concat([df_crossers_medvel_totdist_12, new_cols_rad], axis=1)
df_crossers_medvel_totdist_12_rad

,id,lon,lat,year,month,day,time,hPa,medi_track,FT_MED_VEL_c,...,FT_MED_VEL_v,FT_TOT_DIST_c,FT_TOT_DIST_q,FT_TOT_DIST_v,AT_12h_DIST_c,AT_12h_DIST_q,AT_12h_DIST_v,AT_RAD_DIST_c,AT_RAD_DIST_q,AT_RAD_DIST_v
229,4,0.960,40.700,1979,1,8,18,1016.93,True,0.0,...,31.008022,0.0,0.391,1355.115214,0.0,0.585,293.162,2.0,0.513,1554.441
230,4,1.080,40.761,1979,1,8,19,1016.93,True,0.0,...,31.008022,0.0,0.391,1355.115214,0.0,0.628,313.308,0.0,0.566,1710.421
231,4,1.300,40.746,1979,1,8,20,1016.64,True,0.0,...,31.008022,0.0,0.391,1355.115214,0.0,0.645,321.903,0.0,0.588,1779.949
232,4,1.624,40.699,1979,1,8,21,1016.35,True,0.0,...,31.008022,0.0,0.391,1355.115214,0.0,0.640,319.393,0.0,0.580,1752.835
233,4,1.986,40.693,1979,1,8,22,1015.86,True,0.0,...,31.008022,0.0,0.391,1355.115214,0.0,0.629,313.970,0.0,0.558,1687.237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267292,3807,36.114,44.208,2020,9,8,2,1015.46,True,1.0,...,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN
267293,3807,36.228,44.162,2020,9,8,3,1015.79,True,1.0,...,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN
267294,3807,36.323,44.163,2020,9,8,4,1016.16,True,1.0,...,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN
267295,3807,36.414,44.197,2020,9,8,5,1016.56,True,1.0,...,10.430317,0.0,0.250,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN


### Circle Distance
1. For each ID, find smallest circle around reference point, including 12 consecutive points (radius = max dist to one of 12 consecutive points after reference). Compute for all points of an ID as a reference. Repeat for all IDs.
2. Look for the 10% of reference points with biggest, mid and smallest circles.
3. Find the coord. of the reference points.

In [71]:
# 1. For each ID, find smallest circle around reference point, including 12 consecutive points.
# Vectorize Haversine formula for faster computation
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.deg2rad, [lon1, lat1, lon2, lat2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371
    return c * r

# Compute the maximum distance/smallest circle for each sliding window of 13 points for each id
def circle(df, window_size=13):
    list_circle = []  # To store (id, center_lon, center_lat, max_distance)
    all_dist_circle = []  # To store only the max_distances

    # Group by 'id' to process each id separately
    for id_val, group in df.groupby('id'):
        if len(group) < window_size:
            continue  # Skip this group if it has fewer than `window_size` points
        
        # Iterate over the group with a sliding window of size `window_size`
        for i in range(len(group) - window_size + 1):
            window = group.iloc[i:i + window_size]  # Get the sliding window
            center_lon, center_lat = window.iloc[0][['lon', 'lat']]  # First point as the center
            
            max_distance = 0
            # Compute the maximum distance from the center point to any other point in the window
            for _, row in window.iloc[1:].iterrows():  # Skip the first point since it's the center
                lon, lat = row['lon'], row['lat']
                dist = haversine(center_lon, center_lat, lon, lat)
                max_distance = max(max_distance, dist)
            
            # Append the result as a tuple (id, center_lon, center_lat, max_distance)
            list_circle.append((id_val, center_lon, center_lat, max_distance))
            all_dist_circle.append(max_distance)
    
    return list_circle, all_dist_circle

# Compute the smallest circle for each sliding window of 13 points for each id
list_circle, all_dist_circle = circle(df_crossers)

In [72]:
len(all_dist_circle)

145266

In [73]:
# 2. Look for the 10% of reference points with biggest, mid and smallest circles.
# 2.1. Calculate percentiles
general_max_circle = np.amax(all_dist_circle)                       #calculate the maximum of all circle-distances (upper limit of range)
perc_90_circle = np.percentile(all_dist_circle, 90)                 #calculate the 90 percentile (lower limit of "..10% longest..")

general_median_circle = np.median(all_dist_circle)                  #calculate the median of all circle-distances
perc_45_circle = np.percentile(all_dist_circle, 45)                 #calculate the 45 and 55 percentile (lower and upper limit of"..10% around the median..")
perc_55_circle = np.percentile(all_dist_circle, 55)

general_min_circle = np.amin(all_dist_circle)                       #calculate the minimum of all circle-distances (lower limit of range)
perc_10_circle = np.percentile(all_dist_circle, 10)                 #calculate the 10 percentile (upper limit of "..10% shor..")


In [74]:
# 2.2. Define 10% categories and ref. points
all_bigcircle = []
all_midicircle = []
all_smallcircle = []

ref_big_circle = []
ref_midi_circle = []
ref_small_circle = []

# Loop over each result in the distances list
for result in list_circle:
    ref_loncircle, ref_latcircle, max_dist_circle = result[1], result[2], result[3]
    
    # 10% bigst
    if max_dist_circle >= perc_90_circle:
        all_bigcircle.append(max_dist_circle)
        ref_big_circle.append([ref_loncircle, ref_latcircle])
    
    # midi (45th to 55th percentiles)
    if perc_45_circle <= max_dist_circle <= perc_55_circle:
        all_midicircle.append(max_dist_circle)
        ref_midi_circle.append([ref_loncircle, ref_latcircle])
    
    # 10% smallest
    if max_dist_circle <= perc_10_circle:
        all_smallcircle.append(max_dist_circle)
        ref_small_circle.append([ref_loncircle, ref_latcircle])

In [75]:
# 3. Connect coordinates with all info of cyclones
tracks_bigcircle_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_big_circle])]
tracks_midicircle_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_midi_circle])]
tracks_smallcircle_ref = df_crossers[df_crossers[['lon', 'lat']].apply(tuple, axis=1).isin([tuple(coord) for coord in ref_small_circle])]

#### Preparation for QS table

In [76]:
# 1. All Values
full_dist_circle_values = []
for ID_unique in np.unique(df_crossers.id.values):                       # for each id in df_crossers
    x = np.append(
            np.round(
                np.array(
                    [value[-1:][0] for value in                           # select 4th item, i.e. value
                     [i for i in list_circle if i[0] == ID_unique]]), # select id
                3                                                         # round values up to 3rd decimal place
            ),
            [np.nan] * 12                                                 # append 12 np.nans at the end of cyclone
        )
    full_dist_circle_values.append(x)

AT_CIRCLE_DIST_v = np.concatenate(full_dist_circle_values)
AT_CIRCLE_DIST_v

array([259.569, 279.078, 288.992, ...,     nan,     nan,     nan])

In [77]:
# 1.1. sanity check: compare length of array to make sure same
print(len(df_crossers))
print(len(AT_CIRCLE_DIST_v))

173790
,173790


In [78]:
# 2. All Quantiles
# use rankdata to calculate ranks, with method='average' to handle ties
ranks = rankdata(AT_CIRCLE_DIST_v, method='ordinal', nan_policy='omit')

# calculate quantile ranks
AT_CIRCLE_DIST_q = np.round(
                    np.where(
                        np.isnan(AT_CIRCLE_DIST_v), np.nan,
                            (ranks - 1) / (len(AT_CIRCLE_DIST_v[~np.isnan(AT_CIRCLE_DIST_v)]) - 1)
                    ),
                    3
                )

# this array now contains the quantile ranks
AT_CIRCLE_DIST_q

array([0.573, 0.616, 0.636, ...,   nan,   nan,   nan])

In [79]:
# 3. All Categories (0: Nan, 1: <10%, 2: 45-55%, 3: >90%)
# Assign categories
AT_CIRCLE_DIST_c = np.zeros_like(AT_CIRCLE_DIST_q)

AT_CIRCLE_DIST_c[AT_CIRCLE_DIST_q <= .1] = 1
AT_CIRCLE_DIST_c[(AT_CIRCLE_DIST_q >= .45) & (AT_CIRCLE_DIST_q <= .55)] = 2
AT_CIRCLE_DIST_c[AT_CIRCLE_DIST_q >= .9] = 3

AT_CIRCLE_DIST_c[np.isnan(AT_CIRCLE_DIST_q)] = np.nan
AT_CIRCLE_DIST_c

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [80]:
# 3.1. sanity test: random selection to see whether it worked
print(AT_CIRCLE_DIST_q[2000:2100])
print(AT_CIRCLE_DIST_c[2000:2100])

[0.06  0.065 0.07  0.061 0.035 0.012 0.029 0.042 0.04  0.034 0.037 0.056
, 0.084 0.107 0.114 0.101 0.073 0.04  0.018 0.009 0.018 0.039 0.063 0.084
, 0.113 0.152 0.186 0.195 0.203 0.205 0.197 0.176 0.145 0.122 0.109 0.102
, 0.196 0.301 0.394 0.454 0.478 0.478 0.484 0.551 0.692 0.825   nan   nan
,   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan 0.406 0.388
, 0.354 0.31  0.276 0.26  0.247 0.22  0.187 0.151 0.118 0.095 0.197 0.258
, 0.287 0.314 0.338 0.359 0.374 0.378 0.353 0.273 0.151 0.218 0.338 0.416
, 0.468 0.503 0.531 0.555 0.577 0.595 0.612 0.62  0.621 0.619 0.618 0.618
, 0.617 0.616 0.619 0.624]
,[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  1.
,  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
,  0.  0.  0.  2.  2.  2.  2.  0.  0.  0. nan nan nan nan nan nan nan nan
, nan nan nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
,  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  2.  2.  0.  0.  0.
,  0.  0.  0

In [81]:
# 4. Add Columns to QS Table
new_cols_circle = pd.DataFrame([AT_CIRCLE_DIST_c, AT_CIRCLE_DIST_q, AT_CIRCLE_DIST_v]).T
new_cols_circle.columns = ['AT_CIRCLE_DIST_c', 'AT_CIRCLE_DIST_q', 'AT_CIRCLE_DIST_v']
new_cols_circle.index = df_crossers.index

In [4]:
# 5. QS Table complete
df_crossers_medvel_totdist_12_rad_circle = pd.concat([df_crossers_medvel_totdist_12_rad, new_cols_circle], axis=1)
df_crossers_medvel_totdist_12_rad_circle

<class 'NameError'>: name 'df_crossers_medvel_totdist_12_rad' is not defined

# QS Table

In [84]:
# Rename
df_QS = df_crossers_medvel_totdist_12_rad_circle
df_QS
df_QS.to_csv('/scratch2/mganci/for_marina/data/txt/df_QS.txt', header=True, sep='\t') 

,id,lon,lat,year,month,day,time,hPa,medi_track,FT_MED_VEL_c,...,FT_TOT_DIST_v,AT_12h_DIST_c,AT_12h_DIST_q,AT_12h_DIST_v,AT_RAD_DIST_c,AT_RAD_DIST_q,AT_RAD_DIST_v,AT_CIRCLE_DIST_c,AT_CIRCLE_DIST_q,AT_CIRCLE_DIST_v
229,4,0.960,40.700,1979,1,8,18,1016.93,True,0.0,...,1355.115214,0.0,0.585,293.162,2.0,0.513,1554.441,0.0,0.573,259.569
230,4,1.080,40.761,1979,1,8,19,1016.93,True,0.0,...,1355.115214,0.0,0.628,313.308,0.0,0.566,1710.421,0.0,0.616,279.078
231,4,1.300,40.746,1979,1,8,20,1016.64,True,0.0,...,1355.115214,0.0,0.645,321.903,0.0,0.588,1779.949,0.0,0.636,288.992
232,4,1.624,40.699,1979,1,8,21,1016.35,True,0.0,...,1355.115214,0.0,0.640,319.393,0.0,0.580,1752.835,0.0,0.639,290.526
233,4,1.986,40.693,1979,1,8,22,1015.86,True,0.0,...,1355.115214,0.0,0.629,313.970,0.0,0.558,1687.237,0.0,0.633,287.221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267292,3807,36.114,44.208,2020,9,8,2,1015.46,True,1.0,...,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267293,3807,36.228,44.162,2020,9,8,3,1015.79,True,1.0,...,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267294,3807,36.323,44.163,2020,9,8,4,1016.16,True,1.0,...,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267295,3807,36.414,44.197,2020,9,8,5,1016.56,True,1.0,...,1068.240564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
